# Basketball Tournament Scheduling



In [1]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.install_glpk()

--2024-01-05 15:55:05--  https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6463 (6.3K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   6.31K  --.-KB/s    in 0s      

2024-01-05 15:55:05 (52.1 MB/s) - ‘helper.py’ saved [6463/6463]

Installing idaes via pip...
idaes was successfully installed
Running idaes get-extensions to install Ipopt and k_aug...
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)

[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)

ipopt was successfully installed
k_aug was successfully installed
Installing glpk via apt-get...


In [2]:
## IMPORT LIBRARIES
import pyomo.environ as pyo
import pandas as pd
!apt-get install -y -qq coinor-cbc
!apt-get update
!apt-get install -y glpk-utils


Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 121857 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

### Basketball Tournament Scheduling

Create a round robin style tournament schedule given:  

*    $n_{teams}$ - number of teams in the tournament
*    $n_{G}$ - number of groups, alternatively defined as teams per group
*    $n_{courts}$ - number of courts available
*    $n_{t}$ - number of time slots available

The optimization problem aims to maximize the usage of the courts, while adhering to constraints of the problem:
1.   A court can only be used once in a given time slot
2.   Each team plays against every other team in its group
3.   No team plays a game at the same time slot












In [4]:
%pwd

'/content'

In [3]:
from pyomo.environ import *

def create_basketball_schedule_model(num_teams, num_courts, num_time_slots, teams_per_group):
    # Create a concrete Pyomo model
    model = ConcreteModel()

    # Sets
    model.teams = RangeSet(1, num_teams)
    model.courts = RangeSet(1, num_courts)
    model.time_slots = RangeSet(1, num_time_slots)

    # Integer variables to represent the schedule
    model.x = Var(model.teams, model.teams, model.courts, model.time_slots,  bounds=(0,1), initialize=0.5) # within=Binary,

    # Big-M value # unused currently
    M = 100  # You may need to adjust this based on your problem size

    # Objective function: maximize court and slot usage
    model.obj = Objective(expr=summation(model.x), sense=maximize)

    # Constraint: A court can only be used once in a time slot
    # -- must be less than or equal to 2, as each team (i,j j,i) game is counted twice, if less than or equal to 1, then will be overconstrained
    model.one_game_per_court_per_slot = ConstraintList()
    for c in model.courts:
        for t in model.time_slots:
            model.one_game_per_court_per_slot.add(
                sum(model.x[i, j, c, t] + model.x[j, i, c, t] for i in model.teams for j in model.teams if i != j) <= 2
            )

    # Constraint: Each team plays against another team only once only if they are in the same group, if not in same group, enforce no game played
    team_groups = extract_team_groups(num_teams, teams_per_group)
    model.one_game_per_team_pair = ConstraintList()
    for i in model.teams:
        group_i = find_group_from_teamnum(team_groups, i)
        for j in model.teams:
            group_j = find_group_from_teamnum(team_groups, j)
            if group_i == group_j:
                play_game = 1
            else:
                play_game = 0

            if i != j:
               model.one_game_per_team_pair.add(
                    sum(model.x[i, j, c, t] + model.x[j, i, c, t] for c in model.courts for t in model.time_slots) == play_game
                )

    # Constraint: No team plays a game at the same time slot
    model.no_simultaneous_games = ConstraintList()
    for i in model.teams:
        for t in model.time_slots:
            model.no_simultaneous_games.add(
                sum(model.x[i, j, c, t] + model.x[j, i, c, t] for j in model.teams if i != j for c in model.courts) <= 1
            )

    # # # Constraint: Big-M constraints to enforce binary values
    # model.big_m_constraints = ConstraintList()
    # for i in model.teams:
    #     for j in model.teams:
    #         if i != j:
    #             for c in model.courts:
    #                 for t in model.time_slots:
    #                     # x[i, j, c, t] should be 0 or 1
    #                     model.big_m_constraints.add( M * model.x[i, j, c, t] *  (1 - model.x[i, j, c, t]) == 0)

    return model

def find_group_from_teamnum(team_groups, team_num):
    group_num = -1
    for group, teams in team_groups.items():
        if team_num in teams:
          group_num = group
          return group_num
    return group_num



def extract_team_groups(num_teams, teams_per_group):
    team_groups = {}
    for g in range(1, (num_teams // teams_per_group) + 1):
        teams_in_group = range((g - 1) * teams_per_group + 1, g * teams_per_group + 1)
        team_groups[g] = list(teams_in_group)
    return team_groups

# Example usage with 4 teams, 2 courts, and 2 time slots
num_teams = 25
num_courts = 20
num_time_slots = 6
teams_per_group = 5


# Create the model
model = create_basketball_schedule_model(num_teams, num_courts, num_time_slots, teams_per_group)

# Extract group information
grouping = extract_team_groups(num_teams, teams_per_group)

# Solve the model using the CBC solver

# Get the path to the CBC executable in Colab
cbc_path = !which cbc

# solver = SolverFactory('ipopt')
solver = SolverFactory('glpk')
# Configure Pyomo to use CBC
# solver = SolverFactory('cbc', executable=cbc_path[0])

results = solver.solve(model, tee=False)

# Print the results
print(results)

# Access the solution
for t in model.time_slots:
    for c in model.courts:
        for i in model.teams:
            for j in model.teams:
                if i != j:
                    if round(model.x[i, j, c, t].value) == 1:
                        print(f"Team {i} vs. Team {j} at Court {c} in Time Slot {t}")

# Extract and print the team groups
team_groups = extract_team_groups(num_teams, teams_per_group)
for group, teams in team_groups.items():
    print(f"Group {group}: {teams}")



Problem: 
- Name: unknown
  Lower bound: 3050.0
  Upper bound: 3050.0
  Number of objectives: 1
  Number of constraints: 870
  Number of variables: 75000
  Number of nonzeros: 360000
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 3.8162643909454346
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Team 1 vs. Team 2 at Court 1 in Time Slot 1
Team 3 vs. Team 4 at Court 2 in Time Slot 1
Team 7 vs. Team 9 at Court 3 in Time Slot 1
Team 12 vs. Team 13 at Court 4 in Time Slot 1
Team 16 vs. Team 19 at Court 5 in Time Slot 1
Team 21 vs. Team 23 at Court 6 in Time Slot 1
Team 6 vs. Team 10 at Court 7 in Time Slot 1
Team 11 vs. Team 15 at Court 8 in Time Slot 1
Team 18 vs. Team 20 at Court 9 in Time Slot 1
Team 22 vs. Team 25 at Court 10 in Time Slot 1
Team 1 vs. Team 3 at Court 1 in Time Slot 2
Team 6 vs. Team 